In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [1]:
import tensorflow as tf
from tensorflow import keras
import keras
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D
from keras import backend as K
import tensorflow as tf
import h5py
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
import logging
from keras.models import load_model
#from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.np_utils.to_categorical(y_train, 10)
y_test = keras.utils.np_utils.to_categorical(y_test, 10)

In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
y_train.shape

(50000, 10)

In [ ]:
model = Sequential()
weight_decay = 0.0005
x_shape = [32,32,3]

model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_shape, kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
learning_rate = 0.1
lr_decay = 1e-6
lr_drop = 20

sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
datagen = ImageDataGenerator(
    
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
# (std, mean, and principal components if ZCA whitening is applied).

In [ ]:
def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
    
reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

In [ ]:
batch_size = 128 
maxepoches = 10

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),

                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr], verbose=1)

model.save_weights('Quant_cifar10vgg.h5')

In [ ]:
model.load_weights('Quant_cifar10vgg.h5')
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.738170862197876
Test accuracy: 0.6229000091552734


In [4]:
from keras.layers import Input

In [ ]:
convparams = 320 + 18496
trainable_params = model.count_params()
footprint = trainable_params * 4

#print ("Memory footprint per Image Feed Forward ~= " , footprint / 1024.0 /1024.0 ,"Mb")

In [ ]:
from keras import backend as K

def prepare_softtargets(model,X):
    inp = model.input                                           # input placeholder
    outputs = []
    for layer in model.layers[:]:
        if layer.name == 'flatten_1':
            outputs.append(layer.output)
        if layer.name == 'dense_2':
            outputs.append(layer.output)
            
    functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function
    layer_outs = functor([X, 1.])
    return np.array(layer_outs[0]) , np.array(layer_outs[1])

In [7]:
lastconv_out = []
logit_out = []
for i in range(0,60):
    print("Batch # : ",i)
    l,l2 =  (prepare_softtargets(model,x_train[i*1000:(i+1)*1000]))
    lastconv_out.append(l)
    logit_out.append(l2)
    #lastconv_out , logit_out = prepare_softtargets(model,x_train[i*1000:(i+1)*1000])

lastconv_out = np.array(lastconv_out)
logit_out = np.array(logit_out)
lastconv_out = lastconv_out.reshape((60000 , 9216))
logit_out = logit_out.reshape((60000 , 10))

x_train = 0

Batch # :  0
Batch # :  1
Batch # :  2
Batch # :  3
Batch # :  4
Batch # :  5
Batch # :  6
Batch # :  7
Batch # :  8
Batch # :  9
Batch # :  10
Batch # :  11
Batch # :  12
Batch # :  13
Batch # :  14
Batch # :  15
Batch # :  16
Batch # :  17
Batch # :  18
Batch # :  19
Batch # :  20
Batch # :  21
Batch # :  22
Batch # :  23
Batch # :  24
Batch # :  25
Batch # :  26
Batch # :  27
Batch # :  28
Batch # :  29
Batch # :  30
Batch # :  31
Batch # :  32
Batch # :  33
Batch # :  34
Batch # :  35
Batch # :  36
Batch # :  37
Batch # :  38
Batch # :  39
Batch # :  40
Batch # :  41
Batch # :  42
Batch # :  43
Batch # :  44
Batch # :  45
Batch # :  46
Batch # :  47
Batch # :  48
Batch # :  49
Batch # :  50
Batch # :  51
Batch # :  52
Batch # :  53
Batch # :  54
Batch # :  55
Batch # :  56
Batch # :  57
Batch # :  58
Batch # :  59


In [ ]:
h5f = h5py.File('lastconv_out.h5', 'w')
h5f.create_dataset('dataset_1', data=lastconv_out)
h5f.close()

h5f2 = h5py.File('logit_out.h5', 'w')
h5f2.create_dataset('dataset_1', data=logit_out)
h5f2.close()

In [10]:
lastconv_out = 0
logit_out = 0 

test_lastconv_out = []
test_logit_out = []

for i in range(0,10):
    print("Batch # : ",i)
    l,l2 =  prepare_softtargets(model,x_test[i*1000:(i+1)*1000])
    test_lastconv_out.append(l)
    test_logit_out.append(l2)
    

# lastconv_out.shape , logit_out.shape
test_lastconv_out = np.array(test_lastconv_out)
test_logit_out = np.array(test_logit_out)

test_lastconv_out = test_lastconv_out.reshape((10000 , 9216))
test_logit_out = test_logit_out.reshape((10000 , 10))

Batch # :  0
Batch # :  1
Batch # :  2
Batch # :  3
Batch # :  4
Batch # :  5
Batch # :  6
Batch # :  7
Batch # :  8
Batch # :  9


In [ ]:
h5f = h5py.File('test_lastconv_out.h5', 'w')
h5f.create_dataset('dataset_1', data=test_lastconv_out)
h5f.close()

h5f2 = h5py.File('test_logit_out.h5', 'w')
h5f2.create_dataset('dataset_1', data=test_logit_out)
h5f2.close()

In [16]:
def softmax_c(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis]
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] 
    return e_x / div

In [ ]:
## Logit Regression Method 

results = []

for HiddenNeuron in [2,3,4,5,6,7]:
    
    # Load input,target to studentModel 
    h5f = h5py.File('lastconv_out.h5', 'r')
    lastconv_out = h5f['dataset_1'][:]
    h5f.close()
    
    h5f2 = h5py.File('logit_out.h5', 'r')
    logit_out = h5f2['dataset_1'][:]
    h5f2.close()

    s_model = Sequential()
    s_model.add(Dense(HiddenNeuron,input_dim=9216,activation='relu'))
    s_model.add(Dropout(0.2))
    s_model.add(Dense(num_classes))

    s_model.compile(loss='mse',
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    s_model.fit(lastconv_out, logit_out, epochs=40, verbose=0)
#     s_model.save_weights("student_weights_"+str(HiddenNeuron)+"hidden_0.5_dropout.h5")
    
    compressionRate = model.count_params() / np.float(s_model.count_params()  + convparams)
    
    lastconv_out = 0
    logit_out = 0                
    
    h5f = h5py.File('test_lastconv_out.h5', 'r')
    test_lastconv_out = h5f['dataset_1'][:]
    h5f.close()
    h5f2 = h5py.File('test_logit_out.h5', 'r')
    test_logit_out = h5f2['dataset_1'][:]
    h5f2.close()
    
    pred = s_model.predict(test_lastconv_out)
    probs = softmax_c(pred)
    pred_classes = np.argmax(probs,axis=1)

    accuracy_student = metrics.accuracy_score(y_pred=pred_classes,y_true=np.argmax(y_test,axis=1))
                            
    s_model = 0 
    lastconv_out = 0
    logit_out = 0 
    test_lastconv_out = 0
    test_logit_out = 0 
    
    #results.append(out)

In [17]:
#s_model.save_weights('vgg16_cifar10.h5')